# Proyecto B · Botrytis (Tomate/Uvas) — **Colab Ready** ✅

### Cómo usar (rápido)
1) Sube a `data/raw/` los 4 archivos:  
   - `GSE57586_Botrytis_tomato_raw_reads.txt.gz` + `GSE57586_family.soft.gz`  
   - `GSE57587_Botrytis_grapes_raw_reads.txt.gz` + `GSE57587_family.soft.gz`  
2) Ejecuta todo.  
3) Salidas en `data/processed/` y figuras en `figures/`.  
4) En Power BI usa `expression_long_log2CPM.csv` + `DEG_all.csv`. 

## 0) Setup y utilidades

In [ ]:
try:
    import google.colab  # type: ignore
    IS_COLAB = True
except Exception:
    IS_COLAB = False
from pathlib import Path
BASE = Path("/content/Portafolio-AB") if IS_COLAB else Path("/mnt/data/Portafolio-AB")
RAW, PROC, FIG = BASE/"data/raw", BASE/"data/processed", BASE/"figures"
for d in [RAW, PROC, FIG]: d.mkdir(parents=True, exist_ok=True)

import pandas as pd, numpy as np, gzip
from scipy import stats
import matplotlib.pyplot as plt

def read_gz_table(path: Path, seps=("\t", ",", ";")) -> pd.DataFrame:
    open_fn = gzip.open if str(path).endswith(".gz") else open
    for s in seps:
        try:
            with open_fn(path, "rt", encoding="utf-8", errors="replace") as fh:
                df = pd.read_csv(fh, sep=s, engine="python")
                if df.shape[1] >= 2: return df
        except: pass
    with open_fn(path, "rt", encoding="utf-8", errors="replace") as fh:
        return pd.read_csv(fh, engine="python")

def basic_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(axis=0, how="all").dropna(axis=1, how="all")
    if df.columns[0].lower() not in ["gene","gene_id","id"]:
        df = df.rename(columns={df.columns[0]: "gene"})
    df = df.set_index("gene", drop=True)
    df = df.apply(pd.to_numeric, errors="coerce").fillna(0.0)
    df = df.loc[df.sum(axis=1) > 0]
    return df

def counts_per_million(df: pd.DataFrame) -> pd.DataFrame:
    lib_sizes = df.sum(axis=0).replace(0, np.nan)
    return (df / lib_sizes * 1e6).fillna(0.0)

def bh_fdr(pvals):
    p = np.array(pvals, dtype=float); n = len(p)
    order = np.argsort(p); ranked = p[order]
    q = ranked * n / (np.arange(n)+1)
    q = np.minimum.accumulate(q[::-1])[::-1]
    out = np.empty_like(q); out[order] = q
    return np.clip(out, 0, 1)

def volcano(df, title, outpath, fdr_thresh=0.05, lfc_thresh=1.0):
    plt.figure(figsize=(7,5))
    x = df["log2FC"].values; y = -np.log10(df["pval"].replace(0, np.nextafter(0,1)).values)
    plt.scatter(x, y, s=8, alpha=0.6)
    plt.axhline(-np.log10(fdr_thresh), linestyle="--")
    plt.axvline(lfc_thresh, linestyle="--"); plt.axvline(-lfc_thresh, linestyle="--")
    plt.title(title); plt.xlabel("log2FC"); plt.ylabel("-log10(pval)"); plt.tight_layout()
    plt.savefig(outpath, dpi=150); plt.close()
    print("Guardado:", outpath)

def heatmap_topN(mat_log, topN, title, outpath):
    var = mat_log.var(axis=1).sort_values(ascending=False)
    keep = var.head(min(topN, len(var))).index
    sub = mat_log.loc[keep]
    arr = ((sub.T - sub.mean(axis=1)) / (sub.std(axis=1).replace(0, np.nan))).T.fillna(0.0).values
    plt.figure(figsize=(8, max(4, topN*0.15)))
    plt.imshow(arr, aspect="auto"); plt.colorbar(); plt.title(title); plt.tight_layout()
    plt.savefig(outpath, dpi=150); plt.close()
    print("Guardado:", outpath)

## 1) Cargar, limpiar y normalizar

In [ ]:
tom_raw = read_gz_table(RAW / "GSE57586_Botrytis_tomato_raw_reads.txt.gz")
grp_raw = read_gz_table(RAW / "GSE57587_Botrytis_grapes_raw_reads.txt.gz")

tom = basic_clean(tom_raw); grp = basic_clean(grp_raw)
tom_cpm, grp_cpm = counts_per_million(tom), counts_per_million(grp)
tom_log, grp_log = np.log2(tom_cpm + 1), np.log2(grp_cpm + 1)

tom.to_csv(PROC / "GSE57586_tomate_clean.csv"); grp.to_csv(PROC / "GSE57587_uvas_clean.csv")
tom_log.to_csv(PROC / "GSE57586_tomate_log2CPM.csv"); grp_log.to_csv(PROC / "GSE57587_uvas_log2CPM.csv")

## 2) Definir grupos y DEG

In [ ]:
TOM_CTRL  = ["BB16_tomato_bot"]
TOM_TREAT = ["BB17_tomato_bot", "BB18_tomato_bot"]
GRP_CTRL  = ["Dolce-rnaseq-2-1", "Dolce-rnaseq-2-2"]
GRP_TREAT = ["Dolce-rnaseq-2-3", "Dolce-rnaseq-2-5"]

def pick(all_cols, wanted):
    hits = []
    for w in wanted:
        for c in all_cols:
            if w==c or w in c: hits.append(c)
    out=[]; 
    for c in all_cols:
        if c in hits and c not in out: out.append(c)
    return out

tom_ctrl = pick(tom_log.columns.tolist(), TOM_CTRL); tom_trt = pick(tom_log.columns.tolist(), TOM_TREAT)
grp_ctrl = pick(grp_log.columns.tolist(), GRP_CTRL); grp_trt = pick(grp_log.columns.tolist(), GRP_TREAT)

from scipy import stats
def deg_ttest(log_df, ctrl, trt):
    rows = []
    for g, row in log_df.iterrows():
        x, y = row[ctrl].astype(float).values, row[trt].astype(float).values
        if len(x) >= 1 and len(y) >= 1:
            t, p = stats.ttest_ind(x, y, equal_var=False, nan_policy="omit")
        else:
            t, p = (np.nan, 1.0)
        rows.append((g, (row[trt].mean()-row[ctrl].mean()), p))
    out = pd.DataFrame(rows, columns=["gene","log2FC","pval"]).dropna()
    out["FDR"] = bh_fdr(out["pval"].values)
    return out.sort_values("pval")

deg_tom = deg_ttest(tom_log, tom_ctrl, tom_trt)
deg_uva = deg_ttest(grp_log, grp_ctrl, grp_trt)
deg_tom.to_csv(PROC / "DEG_tomate.csv", index=False)
deg_uva.to_csv(PROC / "DEG_uvas.csv", index=False)
deg_all = pd.concat([deg_tom.assign(dataset='Tomate'), deg_uva.assign(dataset='Uvas')], ignore_index=True)
deg_all.to_csv(PROC / "DEG_all.csv", index=False)

## 3) Figuras (ambos datasets)

In [ ]:
volcano(deg_tom, "Volcano Tomate", FIG/"Volcano_Tomate.png")
volcano(deg_uva, "Volcano Uvas",   FIG/"Volcano_Uvas.png")
heatmap_topN(tom_log, 50, "Heatmap top-50 Tomate (z-score por gen)", FIG/"Heatmap_top50_Tomate.png")
heatmap_topN(grp_log, 50, "Heatmap top-50 Uvas (z-score por gen)",   FIG/"Heatmap_top50_Uvas.png")

## 4) Export BI (long)

In [ ]:
expr_long = pd.concat([
    tom_log.reset_index().melt(id_vars="gene", var_name="sample", value_name="expression").assign(dataset="Tomate"),
    grp_log.reset_index().melt(id_vars="gene", var_name="sample", value_name="expression").assign(dataset="Uvas")
], ignore_index=True)
expr_long.to_csv(PROC / "expression_long_log2CPM.csv", index=False)
print("Listo para BI:", PROC / "expression_long_log2CPM.csv")